<a href="https://colab.research.google.com/github/ShadowofSkull/DevHack2023/blob/main/gcolab/Doc2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install gensim
!pip install spacy
!pip install pandas
!python -m spacy download en_core_web_sm
!pip install numpy

2023-11-04 10:05:48.486067: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-04 10:05:48.486127: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-04 10:05:48.486164: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-04 10:05:50.068784: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 100.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [12]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import gensim
import spacy
import pandas as pd
import numpy as np
from numpy.linalg import norm

In [3]:
#Preprocessing data

job_df = pd.read_csv("drive/MyDrive/Colab Notebooks/data/nyc-jobs.csv")
job_df = job_df[["Business Title", "Job Category", "Job Description", "Minimum Qual Requirements", "Preferred Skills"]]
# salary = job_df[["Salary Range From", "Salary Range To", "Salary Frequency", "Full-Time/Part-Time indicator"]]

# Combining them all into a column
job_df = job_df.astype(str)
job_df["data"] =  job_df[["Business Title", "Job Category", "Job Description", "Minimum Qual Requirements", "Preferred Skills"]].agg(" ".join, axis=1)
job_df.drop(["Business Title", "Job Category", "Job Description", "Minimum Qual Requirements", "Preferred Skills"], axis=1, inplace=True)
print(job_df)

                                                   data
0     Account Manager nan Division of Economic & Fin...
1     EXECUTIVE DIRECTOR, BUSINESS DEVELOPMENT nan T...
2     Maintenance Worker - Technical Services-Heatin...
3     Maintenance Worker - Technical Services-Heatin...
4     Temporary Painter Maintenance & Operations Res...
...                                                 ...
2941  Senior Mechanical Cost Estimator Engineering, ...
2942  Senior Mechanical Cost Estimator Engineering, ...
2943  Conversion Coordinator Building Operations & M...
2944  Conversion Coordinator Building Operations & M...
2945  Administrative Associate Administration & Huma...

[2946 rows x 1 columns]


In [5]:
# Tokenize the documents
nlp = spacy.load('en_core_web_sm')
data = list(job_df['data'])

tokenized_documents = []
for doc in data:
    tokens = [token.text for token in nlp(doc)]
    tokenized_documents.append(tokens)

# Create TaggedDocument objects
tagged_data = [TaggedDocument(words=doc, tags=[str(i)]) for i, doc in enumerate(tokenized_documents)]


In [8]:


# Train the Doc2Vec model
model = Doc2Vec(vector_size=100, window=5, min_count=1, dm=1, epochs=20)

# Build vocabulary
model.build_vocab(tagged_data)

# Train the model on the dataset
model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)

# Save the model if you want to reuse it later
model.save("drive/MyDrive/Colab Notebooks/models/doc2vec_model")



In [57]:
# Model evaluation
resume_df = pd.read_csv("drive/MyDrive/Colab Notebooks/data/Resume.csv")

resumes = list(resume_df["Resume_str"])
print(resumes[310])
resume = resumes[310]
# resume = """I eat burger"""
job_desc = """Detailed knowledge of the Linux OS (RHEL ideally, with Ubuntu knowledge)DBA skills in MySQL (e.g. backups, restores, configurations), other DBs acceptableWebserver configuration (Apache and NGnix)Knowledge of Java based application systems (Tomcat ideal, JBoss and others acceptable)Knowledge of scripting (shell/php/sed/awk/etc..)Willingness to support out of hours and be on call if requiredAWS/Azure/Bare-metal builds.Hudson/Jenkins/Automated build and deploy tools (installation and maintenance)Source Control (SVN/GIT) installation and maintenanceGood written and verbal skillsÂ Useful extra knowledge and skills:Experience with CMS systems (CQ)Programming knowledge (Java ideal)Virtualization (e.g. HyperV)Windows server support and IIS knowledgePostgres DBA skillsKnowledge of Vagrant, Maven, PuppetLDAPSambaKey responsibilities:Maintaining and supporting live sites/machinesMaintaining and supporting development sites/machinesDeployments of code to multiple environments.Pro-active maintenanceTrouble-shooting and diagnosing issuesBuilding new environments.Taking the lead â€“ proposing solutions, not expecting them to be given to you!Writing (where applicable) P1/2 reports.Â Challenges of the roleCoping with multiple, sometimes overlapping work streams â€“ including P1 outagesBleeding edge technology changes Communicating with major stakeholders (TD, MD, CFO, Ops Dir etc..)Proposing technical OS/DB solutions for platforms and sites as required.Making sure the live sites *never* go down!!!Â """


model = Doc2Vec.load('drive/MyDrive/Colab Notebooks/models/doc2vec_model')
v1 = model.infer_vector(resume.split())
v2 = model.infer_vector(job_desc.split())
similarity = 100*(np.dot(np.array(v1), np.array(v2))) / (norm(np.array(v1)) * norm(np.array(v2)))
print(f"{round(similarity, 2)}%")

         CONSULTANT         Experience      Consultant  ,     05/2017   to   Current     Company Name   –   City  ,   State      Managed project schedule and local and remote staff to collect Water Main data and update GIS system.  Develop business process and productivity reports to manage field book scanning project and Water Main data collection project.  Provide Application support for CIS, Infor CMMS, and Neptune AMR applications.  Maintained SharePoint site for I/T department.  Assist Desktop support team with desktop and application user issues.  Developed secure FTP scripts during implementation of CIS and AMR applications after Ransomware attack.         Utility Solutions Consultant  ,     02/2006   to   04/2017     Company Name   –   City  ,   State      Install and configured Cityworks CMMS application, including XML modifications.  Led configuration requirements gathering effort for Cityworks application implementation.  Prepared and provided training for CIS, Meter Reading